# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Flux-Analysis-Tools" data-toc-modified-id="Flux-Analysis-Tools-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Flux Analysis Tools</a></div><div class="lev1 toc-item"><a href="#Yarrowia-SBML-model-from-Chalmers-GEM" data-toc-modified-id="Yarrowia-SBML-model-from-Chalmers-GEM-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Yarrowia SBML model from <a href="https://github.com/djinnome/Yarrowia_lipolytica_W29-GEM" target="_blank">Chalmers GEM</a></a></div><div class="lev1 toc-item"><a href="#Gene-knockouts" data-toc-modified-id="Gene-knockouts-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Gene knockouts</a></div><div class="lev1 toc-item"><a href="#Carbon-sources" data-toc-modified-id="Carbon-sources-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Carbon sources</a></div><div class="lev2 toc-item"><a href="#Malate" data-toc-modified-id="Malate-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Malate</a></div><div class="lev2 toc-item"><a href="#Citrate" data-toc-modified-id="Citrate-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Citrate</a></div><div class="lev2 toc-item"><a href="#Glucose" data-toc-modified-id="Glucose-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Glucose</a></div><div class="lev1 toc-item"><a href="#Simulating-Carbon-sources-across-gene-knockouts" data-toc-modified-id="Simulating-Carbon-sources-across-gene-knockouts-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Simulating Carbon sources across gene knockouts</a></div><div class="lev2 toc-item"><a href="#Growth-rates" data-toc-modified-id="Growth-rates-51"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Growth rates</a></div><div class="lev2 toc-item"><a href="#Fluxes" data-toc-modified-id="Fluxes-52"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Fluxes</a></div><div class="lev1 toc-item"><a href="#Add-symbols-to-genes" data-toc-modified-id="Add-symbols-to-genes-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Add symbols to genes</a></div><div class="lev1 toc-item"><a href="#Yarrowia-measurements" data-toc-modified-id="Yarrowia-measurements-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Yarrowia measurements</a></div><div class="lev2 toc-item"><a href="#Fat-table" data-toc-modified-id="Fat-table-71"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Fat table</a></div><div class="lev2 toc-item"><a href="#Skinny-table" data-toc-modified-id="Skinny-table-72"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Skinny table</a></div><div class="lev2 toc-item"><a href="#Yarrowia-proteins" data-toc-modified-id="Yarrowia-proteins-73"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Yarrowia proteins</a></div><div class="lev2 toc-item"><a href="#Yarrowia-metabolites" data-toc-modified-id="Yarrowia-metabolites-74"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Yarrowia metabolites</a></div><div class="lev2 toc-item"><a href="#Yarrowia-NADPH-ratio" data-toc-modified-id="Yarrowia-NADPH-ratio-75"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Yarrowia NADPH ratio</a></div><div class="lev2 toc-item"><a href="#Yarrowia-growth-rate" data-toc-modified-id="Yarrowia-growth-rate-76"><span class="toc-item-num">7.6&nbsp;&nbsp;</span>Yarrowia growth rate</a></div><div class="lev2 toc-item"><a href="#Metabolite-translation" data-toc-modified-id="Metabolite-translation-77"><span class="toc-item-num">7.7&nbsp;&nbsp;</span>Metabolite translation</a></div>

# Flux Analysis Tools

```
After some discussion with Bill and Scott, I think we’ve assembled a list of simulations to include.  Given that we need these in short order (we want the publication submitted in less than 60 days), I believe we’d be okay with an FBA model rather than including the Boltzman code. Specifically, running the Yarrowia FBA model with glucose, citrate, or malate as the carbon source in combination with wild-type and in silico deletions of idp1, zwf1, mae1. Maximum biomass production should be the objection function since we did the experiments during exponential growth phase.
 
So the 12 conditions are:
Wild-type, Glucose as sole carbon source
Idp1 deletion, Glucose as sole carbon source
Zwf1 deletion, Glucose as sole carbon source
Mae1 deletion, Glucose as sole carbon source
Wild-type, Citrate as sole carbon source
Idp1 deletion, Citrate as sole carbon source
Zwf1 deletion, Citrate as sole carbon source
Mae1 deletion, Citrate as sole carbon source
Wild-type, Malate as sole carbon source
Idp1 deletion, Malate as sole carbon source
Zwf1 deletion, Malate as sole carbon source
Mae1 deletion, Malate as sole carbon source
 
Hopefully these 12 models will yield some interesting eye candy, which can then be compared to the experimental data in our manuscript.  
 
Please let us know if you have questions.  We’d love data as soon as you have it, to synthesize with the paper.
```

In [26]:
import cobra
import cobra.io
from IPython.display import HTML, display, SVG, Markdown, Image
import os, sys
import pandas as pd
from io import StringIO
import numpy as np

def explode(series):
    return pd.concat([pd.DataFrame(list(v), index=np.repeat(k,len(v))) 
            for k,v in series.to_dict().items()
              ])    
def explode2(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

def get_producers( met, fluxes ):
    producers = []
    production_flux = {}
    for rxn in met.reactions:
        if ((fluxes[rxn.id] < 0) and (met in rxn.reactants)) or ((fluxes[rxn.id] > 0) and (met in rxn.products)):
            producers.append( rxn )
            production_flux[rxn.id] = abs(fluxes[rxn.id])
    return producers, pd.Series(production_flux)

def get_consumers( met, fluxes ):
    consumers = []
    consumption_flux = {}
    for rxn in met.reactions:
        if ((fluxes[rxn.id] > 0) and (met in rxn.reactants)) or ((fluxes[rxn.id] < 0) and (met in rxn.products)):
            consumers.append( rxn )
            consumption_flux[rxn.id] = abs(fluxes[rxn.id])
    return consumers, pd.Series(consumption_flux)
def summarize( met, fluxes, out=sys.stdout, use_metabolite_names=True, tol = 1e-5 ):
    ptitle = '\nProducing reactions -- {}[{}] ({})\n'.format(met.name, met.compartment, met.id)
    pcols = 'Pct\tFlux\tRxn ID\tReaction\tReactionIDs\n'
    out.write(ptitle)
    out.write('-'*len(ptitle) + '\n')
    out.write(pcols)
    out.write('\t'.join(['-'*len(col) for col in pcols.split('\t')]) + '\n')
    producers, production_flux = get_producers( met, fluxes )
    production_pct = production_flux/production_flux.sum()*100
    production_pct
    for producer in producers:
        out.write('{:0.3f}\t{:0.3f}\t{}\t{}\t{}\n'.format(production_pct[producer.id], 
                                            production_flux[producer.id], 
                                            producer.id, 
                                            producer.build_reaction_string(use_metabolite_names=True),
                                            producer.build_reaction_string(use_metabolite_names=False)))
    ctitle = '\nConsuming reactions -- {}[{}] ({})\n'.format(met.name, met.compartment, met.id)
    ccols = 'Pct\tFlux\tRxn ID\tReaction\tReactionIDs\n'
    out.write(ctitle)
    out.write('-'*len(ctitle) + '\n')
    out.write(ccols)
    out.write('\t'.join(['-'*len(col) for col in ccols.split('\t')]) + '\n')
    consumers, consumption_flux = get_consumers( met, fluxes )
    consumption_pct = consumption_flux/consumption_flux.sum()*100
    for consumer in consumers:
        out.write('{:0.3f}\t{:0.3f}\t{}\t{}\t{}\n'.format(consumption_pct[consumer.id], 
                                            consumption_flux[consumer.id], 
                                            consumer.id, 
                                            consumer.build_reaction_string(use_metabolite_names=True),
                                            consumer.build_reaction_string(use_metabolite_names=False)))

# Yarrowia SBML model from [Chalmers GEM](https://github.com/djinnome/Yarrowia_lipolytica_W29-GEM)

In [10]:

mdir = 'ModelFiles'
jdir = os.path.join(mdir, 'json')
xdir = os.path.join(mdir, 'xml')
os.listdir('.')
yarrowia = cobra.io.read_sbml_model(os.path.join(xdir,'iYali4.xml' ))
cobra.io.save_json_model(yarrowia, os.path.join(jdir, 'iYali4.json'))

cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xAMINOACID at 0x10eda6128>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xBIOMASS at 0x10f8030f0>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xLIPID at 0x10f803550>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xMAINTENANCE at 0x10f803160>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xPOOL_AC_EM at 0x10f803470>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xPOOL_AC_LP at 0x10f803390>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xPOOL_AC_MM at 0x10f803208>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xPOOL_FA_EM at 0x10f8031d0>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction xPOOL_FA_EN at 0x10f803240>
cobra/io/sbml3.py:

cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003519 at 0x10b2d9780>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003520 at 0x10b2d97f0>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003521 at 0x10b2d98d0>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003522 at 0x10b2d9898>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003523 at 0x10b2d9940>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003524 at 0x10b2d9710>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003525 at 0x10b2d9a20>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003526 at 0x10b2d9a90>
cobra/io/sbml3.py:353 UserWarning: ignoring invalid geneAssociation for <Reaction y003527 at 0x10b2d9b00>
cobra/io/sbml3.py:353 UserWarning: ignoring in

# Gene knockouts

In [11]:
kos = pd.DataFrame()
kos['GeneId'] =pd.Series(dict(
     MAE1 = 'YALI0E18634g',
     GDH1 = 'YALI0F17820g',
     IDP1 = 'YALI0F04095g',
     ZWF1 = 'YALI0E22649g',
     GND1 = 'YALI0B15598g',
    pos5 = 'YALI0E17963g'))
kos.index.name='GeneName'
#display(kos)

kos['GeneObj'] = kos['GeneId'].apply(yarrowia.genes.get_by_id)
kos['RxnObj'] = kos['GeneObj'].apply(lambda x: x.reactions)
kos['RxnId'] = kos['RxnObj'].apply(lambda rxns: [rxn.id for rxn in rxns][0])
kos = kos.reset_index().set_index('GeneId')
kos
#yarrowia.genes

,GeneName,GeneObj,RxnObj,RxnId
GeneId,,,,
YALI0F17820g,GDH1,YALI0F17820g,(y000471: s_0180 + s_0419 + s_0794 + s_1212 --...,y000471
YALI0B15598g,GND1,YALI0B15598g,(y000889: s_0340 + s_1207 --> s_0456 + s_0577 ...,y000889
YALI0F04095g,IDP1,YALI0F04095g,(y000661: s_0942 + s_1211 --> s_0184 + s_0462 ...,y000661
YALI0E18634g,MAE1,YALI0E18634g,(y000719: s_0068 + s_1210 --> s_0460 + s_1214 ...,y000719
YALI0E22649g,ZWF1,YALI0E22649g,(y000466: s_0568 + s_1207 --> s_0335 + s_0794 ...,y000466
YALI0E17963g,pos5,YALI0E17963g,(y000772: s_0437 + s_1205 --> s_0397 + s_0799 ...,y000772


In [6]:
for rxn in kos.loc['YALI0E17963g','RxnObj']:
    display(rxn)

Reaction identifier,y000772
Name,NADH kinase mitochondrial
Memory address,0x01126beb00
Stoichiometry,s_0437 + s_1205 --> s_0397 + s_0799 + s_1214 ATP + NADH --> ADP + H+ + NADPH
GPR,YALI0E17963g or YALI0E23991g or YALI0E27874g
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,y000766
Name,NAD kinase
Memory address,0x01126be710
Stoichiometry,s_0437 + s_1200 --> s_0397 + s_0799 + s_1210 ATP + NAD --> ADP + H+ + NADP(+)
GPR,YALI0E17963g or YALI0E23991g or YALI0E27874g
Lower bound,0.0
Upper bound,1000.0


In [7]:
yarrowia.reactions.y000770

Reaction identifier,y000770
Name,"NADH dehydrogenase, cytosolic/mitochondrial"
Memory address,0x01126bee10
Stoichiometry,s_0794 + s_1203 + s_1537 --> s_1198 + s_1535 H+ + NADH + ubiquinone-6 --> NAD + ubiquinol-6
GPR,YALI0F25135g
Lower bound,0.0
Upper bound,1000.0


In [8]:
yarrowia.reactions.y000769

Reaction identifier,y000769
Name,NAD synthase
Memory address,0x01126be3c8
Stoichiometry,s_0438 + s_0593 + s_0808 + s_1001 <=> s_0425 + s_0637 + s_0800 + s_0994 + s_1201 ATP + deamido-NAD(+) + H2O + L-glutamine <=> AMP + diphosphate + H+ + L-glutamate + NAD
GPR,YALI0A20108g
Lower bound,-1000.0
Upper bound,1000.0


In [9]:
yarrowia.reactions.y000771

Reaction identifier,y000771
Name,NADH kinase
Memory address,0x01126bec88
Stoichiometry,s_0434 + s_1203 --> s_0394 + s_0794 + s_1212 ATP + NADH --> ADP + H+ + NADPH
GPR,YALI0E23991g or YALI0E27874g
Lower bound,0.0
Upper bound,1000.0


In [10]:
yarrowia.reactions.y000772

Reaction identifier,y000772
Name,NADH kinase mitochondrial
Memory address,0x01126beb00
Stoichiometry,s_0437 + s_1205 --> s_0397 + s_0799 + s_1214 ATP + NADH --> ADP + H+ + NADPH
GPR,YALI0E17963g or YALI0E23991g or YALI0E27874g
Lower bound,0.0
Upper bound,1000.0


# Carbon sources


In [12]:
carbon_source_list = """
Exchange reaction	carbon source	formula	MW	carbons	mol C / L	g/L	pH
y001714	glucose	C6H12O6	180.16	6	0.6660746	20	5
y001687	citric acid	C6H8O7	192.12	6	0.6660746	21.3277087	5
y001552	malic acid	C4H6O5	134.09	4	0.6660746	22.32848579	5
"""

carbon_sources = pd.read_table(StringIO(carbon_source_list),index_col='carbon source')
carbon_sources


,Exchange reaction,formula,MW,carbons,mol C / L,g/L,pH
carbon source,,,,,,,
glucose,y001714,C6H12O6,180.16,6,0.666075,20.000000,5
citric acid,y001687,C6H8O7,192.12,6,0.666075,21.327709,5
malic acid,y001552,C4H6O5,134.09,4,0.666075,22.328486,5


## Malate

In [12]:
for rxn in yarrowia.metabolites.s_0067.reactions:
    display(rxn)

Reaction identifier,y001652
Name,alpha-ketoglutarate/malate transporter
Memory address,0x011283d6a0
Stoichiometry,s_0067 + s_0180 <=> s_0066 + s_0181 (S)-malate + 2-oxoglutarate <=> (S)-malate + 2-oxoglutarate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Reaction identifier,y001552
Name,(S)-malate exchange
Memory address,0x011281c6d8
Stoichiometry,s_0067 --> (S)-malate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,y001901
Name,L-malate transport
Memory address,0x0112895ef0
Stoichiometry,s_0067 <=> s_0066 (S)-malate <=> (S)-malate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Citrate

In [13]:
for rxn in yarrowia.metabolites.s_0523.reactions:
    display(rxn)

Reaction identifier,y001687
Name,citrate(3-) exchange
Memory address,0x011284f320
Stoichiometry,s_0523 --> citrate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,y001686
Name,citrate transport
Memory address,0x011284f2b0
Stoichiometry,s_0523 <=> s_0522 citrate <=> citrate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Glucose

In [14]:
for rxn in yarrowia.metabolites.s_0565.reactions:
    display(rxn)
 

Reaction identifier,y001714
Name,D-glucose exchange
Memory address,0x011284fcc0
Stoichiometry,s_0565 <-- D-glucose <--
GPR,
Lower bound,-1.0
Upper bound,-1.0


Reaction identifier,y000370
Name,"exo-1,3-beta-glucan glucohydrase"
Memory address,0x01125d49b0
Stoichiometry,s_0003 + s_0805 --> s_0565 (1-3)-beta-D-glucan + H2O --> D-glucose
GPR,YALI0B03564g or YALI0B17996g or YALI0F05390g
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,y001166
Name,glucose transport
Memory address,0x01127ac048
Stoichiometry,s_0565 --> s_0563 D-glucose --> D-glucose
GPR,YALI0A14212g or YALI0B00396g or YALI0F19184g
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,y001024
Name,sucrose hydrolyzing enxyme
Memory address,0x011273def0
Stoichiometry,s_0805 + s_1466 --> s_0554 + s_0565 H2O + sucrose --> D-fructose + D-glucose
GPR,YALI0E26719g
Lower bound,0.0
Upper bound,1000.0


# Simulating Carbon sources across gene knockouts

## Growth rates

In [13]:
glucose_exchange = 'y001714'
growth_rates = pd.DataFrame()
fluxes = pd.DataFrame()
wild_type = pd.Series()
for carbon_source in carbon_sources.index:  
    with yarrowia:
        yarrowia.reactions.get_by_id(carbon_sources.loc['glucose','Exchange reaction']).lower_bound = 0
        yarrowia.reactions.get_by_id(carbon_sources.loc[carbon_source, 'Exchange reaction']).lower_bound = -1
        wild_type[carbon_source] = yarrowia.optimize().objective_value
        growth_rates[carbon_source] = cobra.flux_analysis.single_gene_deletion(yarrowia,gene_list = kos.index)['growth']
        
growth_rates.index = [list(gene)[0] for gene in growth_rates.index]
growth_rates.index.name = 'GenotypeId'
growth_rates.columns.name = 'Carbon source'
growth_rates['GenotypeName'] = kos['GeneName']

growth_rates.loc['Wild-type'] = wild_type
growth_rates = growth_rates.reset_index().set_index(['GenotypeId','GenotypeName'])
growth_rates.to_csv('Yarrowia_KO_x_carbon_source.csv')
HTML(growth_rates.round(3).to_html())

,Carbon source,glucose,citric acid,malic acid
GenotypeId,GenotypeName,,,
YALI0E17963g,pos5,0.106,0.075,0.051
YALI0E22649g,ZWF1,0.105,0.075,0.050
YALI0E18634g,MAE1,0.104,0.073,0.049
YALI0F04095g,IDP1,0.106,0.074,0.051
YALI0B15598g,GND1,0.105,0.075,0.050
YALI0F17820g,GDH1,0.103,0.073,0.049
Wild-type,NaN,0.106,0.075,0.051


## Fluxes

In [14]:
glucose_exchange = 'y001714'
idx = pd.MultiIndex.from_product([carbon_sources.index, ['wild-type'] + list(kos['GeneName'].values)],names=['carbon source','genotype']  )
#display(idx)
fluxes = pd.DataFrame(columns=idx)
reduced_costs = pd.DataFrame(columns=idx)

for carbon_source in carbon_sources.index:  
    yarrowia.reactions.get_by_id(carbon_sources.loc['glucose','Exchange reaction']).lower_bound = 0
    yarrowia.reactions.get_by_id(carbon_sources.loc[carbon_source, 'Exchange reaction']).lower_bound = -100
    solution = yarrowia.optimize().to_frame()
    fluxes[carbon_source, 'wild-type'] = solution['fluxes']
    reduced_costs[carbon_source, 'wild-type'] = solution['reduced_costs']
    for ko in kos.index:
        with yarrowia:
            yarrowia.genes.get_by_id(ko).knock_out()
            solution = yarrowia.optimize().to_frame()
            fluxes[carbon_source, kos.loc[ko,'GeneName']] = solution['fluxes']
            reduced_costs[carbon_source, kos.loc[ko,'GeneName']] = solution['reduced_costs']
    if carbon_source != 'glucose':
        yarrowia.reactions.get_by_id(carbon_sources.loc['glucose','Exchange reaction']).lower_bound = -100
        yarrowia.reactions.get_by_id(carbon_sources.loc[carbon_source, 'Exchange reaction']).lower_bound = 0    
fluxes['RxnName'] = [yarrowia.reactions.get_by_id(rxn).name for rxn in fluxes.index]
reduced_costs['RxnName'] = [yarrowia.reactions.get_by_id(rxn).name for rxn in reduced_costs.index]
#fluxes['Subsystem'] = [yarrowia.reactions.get_by_id(rxn).notes['SUBSYSTEM'] if 'SUBSYSTEM' in yarrowia.reactions.get_by_id(rxn).notes else '' for rxn in fluxes.index ]
#reduced_costs['Subsystem'] = [yarrowia.reactions.get_by_id(rxn).notes['SUBSYSTEM'] if 'SUBSYSTEM' in yarrowia.reactions.get_by_id(rxn).notes else '' for rxn in reduced_costs.index ]

fluxes.index.name='RxnId'


reduced_costs.index.name='RxnId'
fluxes = fluxes.reset_index().set_index(['RxnId', 'RxnName'])
reduced_costs = reduced_costs.reset_index().set_index(['RxnId', 'RxnName'])
fluxes.to_csv('Yarrowia_fluxes.csv')
reduced_costs.to_csv('Yarrowia_reduced_costs.csv')
display(HTML(fluxes.round(3).to_html()))
display(reduced_costs.round(3))

carbon source                                                     glucose  \
genotype                                                        wild-type   
RxnId        RxnName                                                        
xAMINOACID   Amino acid composition                                 0.000   
xBIOMASS     Biomass production                                    -0.000   
xLIPID       Lipids pool                                            0.000   
xMAINTENANCE Non-growth associated maintenance (NGAM)              -0.014   
xPOOL_AC_EM  Acyl-CoAs pool                                         0.000   
xPOOL_AC_LP  Acyl-CoAs pool                                         0.000   
xPOOL_AC_MM  Acyl-CoAs pool                                         0.000   
xPOOL_FA_EM  Fatty acids pool                                      -0.000   
xPOOL_FA_EN  Fatty acids pool                                       0.000   
xPOOL_FA_LP  Fatty acids pool                                       0.000   
xPOOL_FA_MI  Fatty acids pool                                       0.000   
xPOOL_FA_MM  Fatty acids pool                                       0.000   
y000001      (R)-lactate:ferricytochrome-c 2-oxidoreductase         0.000   
y000002      (R)-lactate:ferricytochrome-c 2-oxidoreductase         0.000   
y000004      (S)-lactate:ferricytochrome-c 2-oxidoreductase         0.000   
y000005      1,3-beta-glucan synthase                               0.000   
y000006      1,6-beta-glucan synthase                              -0.095   
y000007      1-(5-phosphoribosyl)-5-[(5-phosphoribosylamino)...     0.000   
y000008      1-acyl-sn-gylcerol-3-phosphate acyltransferase        -0.000   
y000009      1-acyl-sn-gylcerol-3-phosphate acyltransferase         0.000   
y000010      1-phosphatidylinositol-3-phosphate 5-kinase           -0.012   
y000012      1-pyrroline-5-carboxylate dehydrogenase                0.000   
y000013      2,3-diketo-5-methylthio-1-phosphopentane degrad...     0.000   
y000014      2,5-diamino-6-ribitylamino-4(3H)-pyrimidinone 5...     0.000   
y000015      2,5-diamino-6-ribosylamino-4(3H)-pyrimidinone 5...     0.000   
y000016      2-aceto-2-hydroxybutanoate synthase                   -0.000   
y000017      2-amino-4-hydroxy-6-hydroxymethyldihydropteridi...    -0.007   
y000018      2-aminoadipate transaminase                            0.000   
y000019      2-dehydropantoate 2-reductase                          0.000   
y000020      2-deoxy-D-arabino-heptulosonate 7-phosphate syn...     0.000   
...                                                                   ...   
y300052      glutaryl-CoA dehydrogenase                             0.000   
y300054      crotonyl-CoA hydratase                                 0.000   
y300055      (S)-3-Hydroxybutanoyl-CoA oxidoreductase              -0.458   
y300056      cellobiose glucohydrolase                              0.000   
y300057      L-Glutamate 5-semialdehyde:NAD+ oxidoreductase        -0.000   
y300058      5-oxo-L-proline amidohydrolase (ATP-hydrolysing)       0.000   
y300059      succinyl-CoA:acetoacetate CoA-transferase             -0.000   
y300060      GTP diphosphohydrolase (diphosphate-forming);         -0.014   
y300061      GTP diphosphate-lyase (cyclizing; 3',5'-cyclic-...     0.000   
y300062      L-aspartate 1-carboxy-lyase (beta-alanine-forming)    -0.049   
y300063      glutathione gamma-glutamylaminopeptidase              -0.000   
y300064      3'-Phospho-5'-adenylyl sulfate 3'-phosphohydrolase    -0.012   
y300065      L-Serine:pyruvate aminotransferase                     0.000   
y300066      L-Serine:glyoxylate aminotransferase                   0.000   
y300067      serine racemase                                       -0.000   
y300068      L-Serine hydro-lyase                                  -0.051   
y300069      Uridine triphosphate pyrophosphohydrolase             -0.014   
y300070      Inosine 5'-triphosphate pyrophosphohydrolase          -0.002

In [34]:
gfluxes = fluxes.reset_index().set_index('RxnId')
gfluxes['genes'] = pd.Series([yarrowia.reactions.get_by_id(rxnId).genes 
                                  for rxnId in gfluxes.index],
                             index=gfluxes.index)
with open('genes_of_yarrowia.tsv','w') as out:
    out.write('Gene\n')
    for gene in yarrowia.genes:
        out.write('{}\n'.format(gene.id))
        



In [35]:
!ls -l

total 5688
drwxr-xr-x  3 zuck016  PNL\Domain Users      102 Dec 30 22:32 Figures
-rw-r--r--  1 zuck016  PNL\Domain Users    17067 Dec 22 17:07 LICENSE.md
drwxr-xr-x  5 zuck016  PNL\Domain Users      170 Dec 30 21:38 Measurements
drwxr-xr-x  9 zuck016  PNL\Domain Users      306 Jan  2 10:20 ModelFiles
-rw-r--r--  1 zuck016  PNL\Domain Users    16731 Dec 22 17:07 README.md
drwxr-xr-x@ 3 zuck016  PNL\Domain Users      102 Feb 28 15:53 Xrefs
-rw-r--r--  1 zuck016  PNL\Domain Users      595 Feb 28 16:56 Yarrowia_KO_x_carbon_source.csv
-rw-r--r--  1 zuck016  PNL\Domain Users   380847 Feb 28 16:56 Yarrowia_fluxes.csv
-rw-r--r--  1 zuck016  PNL\Domain Users  1941355 Feb 28 15:48 Yarrowia_nutrient_ko_simulations.ipynb
-rw-r--r--  1 zuck016  PNL\Domain Users   512700 Feb 28 16:56 Yarrowia_reduced_costs.csv
-rw-r--r--  1 zuck016  PNL\Domain Users    14602 Dec 30 22:12 enzyme_activities_of_itaconate.tab
-rw-r--r--  1 zuck016  PNL\Domain Users    10994 Feb 28 17:09 genes_of_yarrowia.tsv


# Add symbols to genes


In [21]:
import pandas as pd
proteins = pd.read_table('yarrowia/avg_protein_abundance.txt',index_col='gene')
proteins[proteins['pathway'].str.contains(r'nitrogen|pentose phosphate|citric acid cycle|glycolysis|PPP')]

,symbol,pathway,Cit_idp,Cit_mae,Cit_WT,Cit_zwf,Glu_idp,Glu_mae,Glu_WT,Glu_zwf,Mal_idp,Mal_mae,Mal_WT,Mal_zwf
gene,,,,,,,,,,,,,,
YALI0D09361g,ACO1,citric acid cycle,74.446667,69.831667,56.690000,50.938333,85.488333,85.150000,74.923333,39.193333,135.201667,43.688333,54.481667,61.476667
YALI0E00638g,CIT1,citric acid cycle,130.285000,128.388333,132.106667,93.611667,102.581667,140.891667,148.475000,68.400000,91.938333,8.171667,64.815000,38.155000
YALI0E02684g,CIT2,citric acid cycle,47.720000,67.676667,60.515000,34.350000,48.258333,82.786667,88.900000,33.538333,89.101667,40.323333,68.751667,51.346667
YALI0C06776g,FUM1,citric acid cycle,26.165000,36.101667,32.863333,22.515000,29.866667,31.196667,31.601667,18.200000,62.038333,27.496667,36.745000,39.255000
YALI0E05137g,IDH1,citric acid cycle,32.436667,43.623333,34.053333,30.973333,71.000000,58.091667,57.841667,30.518333,65.535000,44.126667,53.658333,34.775000
YALI0D06303g,IDH2,citric acid cycle,22.601667,35.640000,32.863333,18.871667,65.163333,58.313333,72.245000,16.088333,32.403333,7.773333,36.901667,10.388333
YALI0F04095g,IDP1,citric acid cycle,0.040000,66.350000,47.326667,24.016667,0.116667,21.573333,26.948333,27.550000,0.283333,52.923333,53.761667,62.566667
YALI0E24013g,ISC1,citric acid cycle,33.313333,70.426667,44.888333,25.588333,24.193333,18.883333,29.233333,16.126667,23.325000,8.138333,32.206667,13.618333
YALI0D04741g,ISC2,citric acid cycle,4.110000,12.551667,7.385000,3.155000,7.858333,5.195000,7.706667,4.491667,7.870000,1.566667,5.650000,3.493333


In [2]:
kos = """YALI0F04095g
YALI0E18634g
YALI0F17820g
YALI0E22649g
YALI0B15598g
YALI0E17963g""".split('\n')


# Yarrowia measurements

## Fat table

In [2]:
import pandas as pd
from IPython.display import HTML
yarrowia_data = pd.read_excel('Measurements/yarrowia_nadph_C13_metabolomics_SRM_proteomics_master_sheet_170320.xlsx',sheet_name='DATA by sample')
HTML(yarrowia_data.to_html())

,Tube label,Strain,Genotype,Carbon source,Replicate (2 technical reps have been collected for each of these replicates),dry weight (g/L),NADPH ratio (NADP+:NADPH),SRM (fmole/ug) - average of peptides,YALI0B08536g ACA1,YALI0E11099g ACA2,YALI0E34793g ACL1,YALI0D24431g ACL2,YALI0F05962g ACS1,YALI0D23683g LAT1,YALI0E03212g LDH1,YALI0C06446g LDH2,YALI0D20768g LPD1,YALI0F20702g PDA1,YALI0E27005g PDB1,YALI0D09361g ACO1,YALI0E00638g CIT1,YALI0E02684g CIT2,YALI0C06776g FUM1,YALI0E05137g IDH1,YALI0D06303g IDH2,YALI0F04095g IDP1,YALI0E24013g ISC1,YALI0D04741g ISC2,YALI0E33517g KGD1,YALI0E16929g KGD2,YALI0D16753g MDH1,YALI0D11374g SDH1,YALI0D23397g SDH2,YALI0E29667g SDH3,YALI0A14784g SDH4,YALI0C11407g ACC1,YALI0F30679g CEM1,YALI0B15059g FAS1,YALI0B19382g FAS2,YALI0C16995g PCK1,YALI0B22308g HXK1,YALI0E15488g HXK2,YALI0D16357g PFK1,YALI0F09185g PYK1,YALI0F05214g TPI1,YALI0F16819g ENO1,YALI0E26004g FBA1,YALI0A15972g FBP1,YALI0B02728g GPM1,YALI0E18634g MAE1,YALI0E14190g MDH2,YALI0F07711g PGI1,YALI0D12400g PGK1,YALI0C24101g PYC2,YALI0C06369g TDH1,YALI0C16885g ICL1,YALI0D19140g MLS1,YALI0E15708g MLS2,YALI0F17820g GDH1,YALI0E09603g GDH2,YALI0F00506g GLN1,YALI0D13024g GLN2,YALI0B19998g GLT1,YALI0F07821g GLK,YALI0B14927g RBK1,YALI0C19085g SOL1,YALI0E11671g SOL4,YALI0E22649g ZWF1,YALI0A21263g ALO,YALI0F30811g ARA2,YALI0D02321g UGD,YALI0F10923g XLK,YALI0E12463g XYD,YALI0D07634g XYR,YALI0B15598g GND1,YALI0B06941g RKI1,YALI0C11880g RPE1,YALI0F15587g TAL1,YALI0E06479g TKL1,YALI0E02090g PGM1,YALI0E27742g GCN4,YALI0E24937g STP1,YALI0B05478g STP2,YALI0E07942g CREA,YALI0C14740p GRR1,YALI0C05665g BMH1,YALI0B14377p BMH2,YALI0C07821g COL26,YALI0F11979g RTG3,YALI0D15334g TYE7/SRE1,YALI0D12628g FARA,YALI0E03410g ERT1,YALI0C19151g CAT8,YALI0E16236p STE12,YALI0C14784g OPI1,YALI0E16577g ASH1,YALI0C22682g GATA_A,YALI0D20482g GATA_B,YALI0E02442g GATA_C,YALI0F17886g GATA_H,YALI0E30547g SNT2,YALI0E05555g SRE,YALI0F05346g WC2,Metabolites (ng/mg DW),2-oxoglutarate,Arabitol,Citrate,Erythrose-4P,Fructose-6P,Fumarate,Glucose-6P,Glutamate,Glutamine,Glyceraldehyde-3P,Glycerol,Glycine,Isocitrate,Malate,Mannitol,PEP,Pyruvate,Ribose-5P,Ribulose-5P,Sedoheptulose-7P,Serine,Succinate,Xylulose-5P
0,IDP CIT 1,FEB189,idp1,citric acid,1,0.246667,16.504587,NaN,14.730,29.390,11.740,120.585,51.380,46.430,0.565,30.235,16.660,34.040,17.660,68.200,120.490,47.935,24.115,32.105,19.735,0.055,30.890,3.410,32.740,52.815,92.830,12.990,25.040,6.460,2.610,20.665,0.475,0.460,5.980,384.660,0.365,0.355,8.170,67.165,127.005,1678.640,53.345,0.725,30.015,0.150,222.265,83.350,1.160,13.480,121.155,374.370,7.010,40.800,32.085,25.730,10.970,0.040,12.665,7.420,0.480,0.216667,0.215,4.025,0.445,0.495,0.035,0.345,0.065,43.215,183.110,9.825,0.220000,10.815,47.780,7.070,0.760,0.580,0.406667,0.565,0.436667,66.840,76.700,0.056667,1.483333,0.710,0.290,0.180000,0.1875,0.033333,1.056667,0.140000,0.165,0.1400,0.046,0.193333,0.146667,0.075,0.070,NaN,3752.158581,1682.160568,40963.387919,19.942003,183.215540,508.806418,182.589015,40.653685,67019.980823,34.334111,151.871686,298.804998,289.019694,4551.105046,11849.673923,174.492247,1884.467193,518.593802,648.757143,297.036426,1172.811004,5021.299785,1586.128654
1,IDP CIT 2,FEB189,idp1,citric acid,2,0.223333,4.139212,NaN,14.015,28.495,12.235,131.645,54.645,92.340,1.230,30.325,17.830,38.575,18.210,70.415,136.270,56.855,24.880,31.270,25.855,0.025,32.830,4.205,36.735,53.650,93.570,13.845,26.035,7.650,2.970,20.095,0.525,0.475,5.540,437.335,0.440,0.520,8.600,64.455,117.610,1512.490,46.580,2.095,32.035,0.145,220.470,108.155,3.380,15.405,195.015,367.665,8.850,55.490,34.010,31.675,14.220,0.050,14.720,6.395,0.675,0.480000,0.190,4.005,0.440,0.525,0.050,0.290,0.200,41.115,182.300,9.955,0.220000,13.660,51.400,8.805,0.630,0.950,0.500000,0.590,0.523333,43.335,82.085,0.043333,1.760000,0.795,0.335,0.193333,0.2375,0.036667,1.580000,0.153333,0.260,0.1900,0.034,0.226667,0.150000,0.080,0.140,NaN,2967.321150,1549.776940,43050.444214,22.324506,200.005025,456.150133,200.816492,37.665463,80087.83327

In [3]:
proteins = yarrowia_data.columns[8:-25]
metabolites = yarrowia_data.columns[-23:]
dry_weight = yarrowia_data.columns[5]
nadph_ratio = yarrowia_data.columns[6]
print('\n'.join(metabolites))

2-oxoglutarate
Arabitol
Citrate
Erythrose-4P
Fructose-6P
Fumarate
Glucose-6P
Glutamate
Glutamine
Glyceraldehyde-3P
Glycerol
Glycine
Isocitrate
Malate
Mannitol
PEP
Pyruvate
Ribose-5P
Ribulose-5P
Sedoheptulose-7P
Serine
Succinate
Xylulose-5P


In [4]:
id_vars=['Tube label', 'Strain', 'Genotype', 'Carbon source',
       'Replicate (2 technical reps have been collected for each of these replicates)']
biological_replicate = id_vars[1:-1]
biological_replicate

['Strain', 'Genotype', 'Carbon source']

## Skinny table

In [5]:
skinny_yarrowia = yarrowia_data.melt(id_vars=id_vars).set_index(id_vars)
skinny_yarrowia

,,,,,variable,value
Tube label,Strain,Genotype,Carbon source,Replicate (2 technical reps have been collected for each of these replicates),,
IDP CIT 1,FEB189,idp1,citric acid,1,dry weight (g/L),0.246667
IDP CIT 2,FEB189,idp1,citric acid,2,dry weight (g/L),0.223333
IDP CIT 3,FEB189,idp1,citric acid,3,dry weight (g/L),0.243333
mae CIT 1,FEB188,mae1,citric acid,1,dry weight (g/L),0.180000
mae CIT 2,FEB188,mae1,citric acid,2,dry weight (g/L),0.173333
mae CIT 3,FEB188,mae1,citric acid,3,dry weight (g/L),0.193333
WT CIT 1,FKP391,wild-type,citric acid,1,dry weight (g/L),0.090000
WT CIT 2,FKP391,wild-type,citric acid,2,dry weight (g/L),0.123333
WT CIT 3,FKP391,wild-type,citric acid,3,dry weight (g/L),0.110000


## Yarrowia proteins

In [6]:
yarrowia_proteins = yarrowia_data.melt(id_vars=id_vars,value_vars=proteins, var_name='proteins', value_name=yarrowia_data.columns[7]).set_index(id_vars)
yarrowia_proteins[yarrowia_proteins['proteins'] == 'YALI0A14784g SDH4']

,,,,,proteins,SRM (fmole/ug) - average of peptides
Tube label,Strain,Genotype,Carbon source,Replicate (2 technical reps have been collected for each of these replicates),,
IDP CIT 1,FEB189,idp1,citric acid,1,YALI0A14784g SDH4,2.610
IDP CIT 2,FEB189,idp1,citric acid,2,YALI0A14784g SDH4,2.970
IDP CIT 3,FEB189,idp1,citric acid,3,YALI0A14784g SDH4,2.375
mae CIT 1,FEB188,mae1,citric acid,1,YALI0A14784g SDH4,1.890
mae CIT 2,FEB188,mae1,citric acid,2,YALI0A14784g SDH4,3.435
mae CIT 3,FEB188,mae1,citric acid,3,YALI0A14784g SDH4,3.285
WT CIT 1,FKP391,wild-type,citric acid,1,YALI0A14784g SDH4,2.765
WT CIT 2,FKP391,wild-type,citric acid,2,YALI0A14784g SDH4,2.725
WT CIT 3,FKP391,wild-type,citric acid,3,YALI0A14784g SDH4,3.355


## Yarrowia metabolites

In [7]:
yarrowia_metabolites = yarrowia_data.melt(id_vars=id_vars,
                                          value_vars=metabolites, 
                                          var_name='metabolites', value_name=yarrowia_data.columns[-24]).set_index(id_vars)
yarrowia_metabolites

,,,,,metabolites,Metabolites (ng/mg DW)
Tube label,Strain,Genotype,Carbon source,Replicate (2 technical reps have been collected for each of these replicates),,
IDP CIT 1,FEB189,idp1,citric acid,1,2-oxoglutarate,3752.158581
IDP CIT 2,FEB189,idp1,citric acid,2,2-oxoglutarate,2967.321150
IDP CIT 3,FEB189,idp1,citric acid,3,2-oxoglutarate,2837.314151
mae CIT 1,FEB188,mae1,citric acid,1,2-oxoglutarate,6965.776128
mae CIT 2,FEB188,mae1,citric acid,2,2-oxoglutarate,4620.059834
mae CIT 3,FEB188,mae1,citric acid,3,2-oxoglutarate,3767.991609
WT CIT 1,FKP391,wild-type,citric acid,1,2-oxoglutarate,3856.512402
WT CIT 2,FKP391,wild-type,citric acid,2,2-oxoglutarate,3092.003281
WT CIT 3,FKP391,wild-type,citric acid,3,2-oxoglutarate,4058.666822


In [8]:
avg_yarrowia_metabolites = yarrowia_metabolites.groupby(biological_replicate + ['metabolites']).mean()
HTML(avg_yarrowia_metabolites.xs('wild-type', 
                                 level='Genotype', 
                                 drop_level=False).to_html())

In [13]:
import numpy as np
std_yarrowia_metabolites = yarrowia_metabolites.groupby(biological_replicate + ['metabolites']).std()
error_yarrowia_metabolites = std_yarrowia_metabolites.join(avg_yarrowia_metabolites,lsuffix='_std', rsuffix='_mean')
error_yarrowia_metabolites['$\mu+\sigma$'] = avg_yarrowia_metabolites + std_yarrowia_metabolites
error_yarrowia_metabolites['$\mu-\sigma$'] = avg_yarrowia_metabolites - std_yarrowia_metabolites
error_yarrowia_metabolites[r'$\log\left(\mu+\sigma\right)$']= error_yarrowia_metabolites[r'$\mu+\sigma$'].apply(np.log)
error_yarrowia_metabolites[r'$\log\left(\mu-\sigma\right)$']= error_yarrowia_metabolites[r'$\mu-\sigma$'].apply(np.log)
error_yarrowia_metabolites

Metabolites (ng/mg DW)_std  \
Strain Genotype  Carbon source metabolites                                     
FEB184 zwf1      citric acid   2-oxoglutarate                     194.341456   
                               Arabitol                            33.107456   
                               Citrate                           1371.458463   
                               Erythrose-4P                         7.292727   
                               Fructose-6P                         29.809264   
                               Fumarate                             9.419589   
                               Glucose-6P                          18.342471   
                               Glutamate                            2.293175   
                               Glutamine                         6022.531691   
                               Glyceraldehyde-3P                    5.062709   
                               Glycerol                            15.720708   
                               Glycine                             11.941219   
                               Isocitrate                          72.964035   
                               Malate                            1055.344845   
                               Mannitol                           968.427725   
                               PEP                                111.795746   
                               Pyruvate                           657.856384   
                               Ribose-5P                           30.293247   
                               Ribulose-5P                        167.908718   
                               Sedoheptulose-7P                    14.880617   
                               Serine                              23.219262   
                               Succinate                          742.009588   
                               Xylulose-5P                         64.700126   
                 glucose       2-oxoglutarate                    4567.280940   
                               Arabitol                           890.860360   
                               Citrate                          32479.698607   
                               Erythrose-4P                       112.697653   
                               Fructose-6P                        810.512884   
                               Fumarate                           807.488924   
                               Glucose-6P                        1193.729925   
...                                                                      ...   
FKP391 wild-type glucose       Pyruvate                           462.807824   
                               Ribose-5P                          187.921363   
                               Ribulose-5P                        307.719916   
                               Sedoheptulose-7P                   124.764477   
                               Serine                             391.767888   
                               Succinate                          934.680684   
                               Xylulose-5P                        573.465413   
                 malic acid    2-oxoglutarate                    2186.965948   
                               Arabitol                          2480.390026   
                               Citrate                          10580.112328   
                               Erythrose-4P                         2.724040   
                               Fructose-6P                        102.057594   
                               Fumarate                           530.760385   
                               Glucose-6P                          98.135618   
                               Glutamate                            4.719613   
                               Glutamine                        42181.592463   
                               Glyceraldehyde-3P                    9.454069   
                               Glycerol           

## Yarrowia NADPH ratio

In [51]:
yarrowia_nadph_ratio = yarrowia_data.melt(id_vars=id_vars,value_vars=nadph_ratio, var_name='NADP(H)_ratio', value_name=nadph_ratio).set_index(id_vars)
yarrowia_nadph_ratio

,,,,,NADP(H)_ratio,NADPH ratio (NADP+:NADPH)
Tube label,Strain,Genotype,Carbon source,Replicate (2 technical reps have been collected for each of these replicates),,
IDP CIT 1,FEB189,idp1,citric acid,1,NADPH ratio (NADP+:NADPH),16.504587
IDP CIT 2,FEB189,idp1,citric acid,2,NADPH ratio (NADP+:NADPH),4.139212
IDP CIT 3,FEB189,idp1,citric acid,3,NADPH ratio (NADP+:NADPH),15.581348
mae CIT 1,FEB188,mae1,citric acid,1,NADPH ratio (NADP+:NADPH),0.682935
mae CIT 2,FEB188,mae1,citric acid,2,NADPH ratio (NADP+:NADPH),5.594196
mae CIT 3,FEB188,mae1,citric acid,3,NADPH ratio (NADP+:NADPH),3.533946
WT CIT 1,FKP391,wild-type,citric acid,1,NADPH ratio (NADP+:NADPH),0.492320
WT CIT 2,FKP391,wild-type,citric acid,2,NADPH ratio (NADP+:NADPH),0.388943
WT CIT 3,FKP391,wild-type,citric acid,3,NADPH ratio (NADP+:NADPH),0.504554


In [52]:
avg_yarrowia_nadph_ratio = yarrowia_nadph_ratio.groupby(biological_replicate + ['NADP(H)_ratio']).mean()
avg_yarrowia_nadph_ratio

NADPH ratio (NADP+:NADPH)
Strain Genotype  Carbon source NADP(H)_ratio                                       
FEB184 zwf1      citric acid   NADPH ratio (NADP+:NADPH)                   0.490602
                 glucose       NADPH ratio (NADP+:NADPH)                  13.802420
                 malic acid    NADPH ratio (NADP+:NADPH)                   7.220769
FEB188 mae1      citric acid   NADPH ratio (NADP+:NADPH)                   3.270359
                 glucose       NADPH ratio (NADP+:NADPH)                   2.597845
                 malic acid    NADPH ratio (NADP+:NADPH)                   1.677965
FEB189 idp1      citric acid   NADPH ratio (NADP+:NADPH)                  12.075049
                 glucose       NADPH ratio (NADP+:NADPH)                   2.128674
                 malic acid    NADPH ratio (NADP+:NADPH)                   3.563399
FKP391 wild-type citric acid   NADPH ratio (NADP+:NADPH)                   0.461939
                 glucose       NADPH ratio (NADP+:NADPH)                   2.048413
                 malic acid    NADPH ratio (NADP+:NADPH)                   2.035706

In [43]:
yarrowia_data.columns[-24]

'Metabolites (ng/mg DW)'

## Yarrowia growth rate

In [53]:
yarrowia_growth_rate = yarrowia_data.melt(id_vars=id_vars,value_vars=dry_weight, var_name='Dry_weight', value_name=dry_weight).set_index(id_vars)
yarrowia_growth_rate

,,,,,Dry_weight,dry weight (g/L)
Tube label,Strain,Genotype,Carbon source,Replicate (2 technical reps have been collected for each of these replicates),,
IDP CIT 1,FEB189,idp1,citric acid,1,dry weight (g/L),0.246667
IDP CIT 2,FEB189,idp1,citric acid,2,dry weight (g/L),0.223333
IDP CIT 3,FEB189,idp1,citric acid,3,dry weight (g/L),0.243333
mae CIT 1,FEB188,mae1,citric acid,1,dry weight (g/L),0.180000
mae CIT 2,FEB188,mae1,citric acid,2,dry weight (g/L),0.173333
mae CIT 3,FEB188,mae1,citric acid,3,dry weight (g/L),0.193333
WT CIT 1,FKP391,wild-type,citric acid,1,dry weight (g/L),0.090000
WT CIT 2,FKP391,wild-type,citric acid,2,dry weight (g/L),0.123333
WT CIT 3,FKP391,wild-type,citric acid,3,dry weight (g/L),0.110000


In [55]:
avg_yarrowia_growth_rate= yarrowia_growth_rate.groupby(biological_replicate + ['Dry_weight']).mean()
avg_yarrowia_growth_rate

dry weight (g/L)
Strain Genotype  Carbon source Dry_weight                        
FEB184 zwf1      citric acid   dry weight (g/L)          0.324444
                 glucose       dry weight (g/L)          0.078889
                 malic acid    dry weight (g/L)          0.141111
FEB188 mae1      citric acid   dry weight (g/L)          0.182222
                 glucose       dry weight (g/L)          0.162222
                 malic acid    dry weight (g/L)          0.268889
FEB189 idp1      citric acid   dry weight (g/L)          0.237778
                 glucose       dry weight (g/L)          0.155556
                 malic acid    dry weight (g/L)          0.217778
FKP391 wild-type citric acid   dry weight (g/L)          0.107778
                 glucose       dry weight (g/L)          0.097778
                 malic acid    dry weight (g/L)          0.135556

## Metabolite translation

In [1]:
print('\n'.join(metabolites))

NameError: name 'metabolites' is not defined

In [69]:
%%writefile metabolite_translation.tab
success		BioCyc:2-KETOGLUTARATE		2-oxoglutarate		RefMet:Oxoglutaric acid		IAF1260:akg		MetaboLights:MTBLC16810		HMDB:HMDB00208		ChemSpider:144236		ChEBI:16810		PubChem:164533		KNApSAcK:C00000769		KEGG:C00026		CAS:328-50-7	InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(H,7,8)(H,9,10)/p-2
unknown		Arabitol
success		BioCyc:CIT		citrate		RefMet:Citric acid		IAF1260:cit		MetaboLights:MTBLC16947		HMDB:HMDB00094		ChemSpider:29081		PubChem:31348		KNApSAcK:C00007619		ChEBI:16947		KEGG:C00158		CAS:77-92-9	InChI=1S/C6H8O7/c7-3(8)1-6(13,5(11)12)2-4(9)10/h13H,1-2H2,(H,7,8)(H,9,10)(H,11,12)/p-3
success		BioCyc:ERYTHROSE-4P		D-erythrose 4-phosphate		RefMet:Erythrose 4-phosphate		IAF1260:e4p		MetaboLights:MTBLC16897		HMDB:HMDB01321		ChemSpider:4573609		ChEBI:16897		PubChem:5459862		KEGG:C00279		CAS:585-18-2	InChI=1S/C4H9O7P/c5-1-3(6)4(7)2-11-12(8,9)10/h1,3-4,6-7H,2H2,(H2,8,9,10)/p-2/t3-,4+/m0/s1
success		BioCyc:FRUCTOSE-6P		beta-D-fructofuranose 6-phosphate		IAF1260:f6p		ChemSpider:10239172		ChEBI:57634		PubChem:21604863		CAS:643-13-0	InChI=1S/C6H13O9P/c7-2-6(10)5(9)4(8)3(15-6)1-14-16(11,12)13/h3-5,7-10H,1-2H2,(H2,11,12,13)/p-2/t3-,4-,5+,6-/m1/s1
success		BioCyc:FUM		fumarate		RefMet:Fumaric acid		IAF1260:fum		MetaboLights:MTBLC29806		HMDB:HMDB00134		DrugBank:DB01677		ChemSpider:4573886		KNApSAcK:C00001183		PubChem:5460307		ChEBI:29806		KEGG:C00122		CAS:110-17-8	InChI=1S/C4H4O4/c5-3(6)1-2-4(7)8/h1-2H,(H,5,6)(H,7,8)/p-2/b2-1+
unknown		Glucose-6P
success		BioCyc:Glutamates		glutamate	InChI=1S/C5H9NO4/c6-3(5(9)10)1-2-4(7)8/h3H,1-2,6H2,(H,7,8)(H,9,10)/p-1
success		BioCyc:GLN		L-glutamine		RefMet:Glutamine		IAF1260:gln__L		MetaboLights:MTBLC58359		HMDB:HMDB00641		ChEBI:58359		PubChem:6992086		KEGG:C00064		CAS:56-85-9	InChI=1S/C5H10N2O3/c6-3(5(9)10)1-2-4(7)8/h3H,1-2,6H2,(H2,7,8)(H,9,10)/t3-/m0/s1
success		BioCyc:GAP		D-glyceraldehyde 3-phosphate		RefMet:Glyceraldehyde-3-phosphate		IAF1260:g3p		HMDB:HMDB01112		KEGG:C00118		CAS:591-57-1		ChEBI:59776		PubChem:24794350	InChI=1S/C3H7O6P/c4-1-3(5)2-9-10(6,7)8/h1,3,5H,2H2,(H2,6,7,8)/p-2/t3-/m0/s1
success		BioCyc:GLYCEROL		glycerol		RefMet:Glycerol		IAF1260:glyc		MetaboLights:MTBLC17754		HMDB:HMDB00131		DrugBank:DB04077		ChemSpider:733		PubChem:753		ChEBI:17754		KEGG:C00116		CAS:56-81-5	InChI=1S/C3H8O3/c4-1-3(6)2-5/h3-6H,1-2H2
success		BioCyc:GLY		glycine		ChEBI:57305		RefMet:Glycine		IAF1260:gly		MetaboLights:MTBLC15428		HMDB:HMDB00123		DrugBank:DB00145		ChemSpider:730		PubChem:5257127		KNApSAcK:C00001361		KEGG:C00037		CAS:56-40-6	InChI=1S/C2H5NO2/c3-1-2(4)5/h1,3H2,(H,4,5)
ambiguous	BioCyc:Isocitrate		BioCyc:THREO-DS-ISO-CITRATE	
success		BioCyc:RS-Malate		(RS)-malate		ChEBI:15595	InChI=1S/C4H6O5/c5-2(4(8)9)1-3(6)7/h2,5H,1H2,(H,6,7)(H,8,9)/p-2
success		BioCyc:MANNITOL		D-mannitol		RefMet:Mannitol		IAF1260:mnl		MetaboLights:MTBLC16899		HMDB:HMDB00765		DrugBank:DB00742		ChemSpider:6015		ChEBI:16899		PubChem:6251		KEGG:C00392		CAS:69-65-8	InChI=1S/C6H14O6/c7-1-3(9)5(11)6(12)4(10)2-8/h3-12H,1-2H2/t3-,4-,5-,6-/m1/s1
success		BioCyc:PHOSPHO-ENOL-PYRUVATE		phosphoenolpyruvate		RefMet:Phosphoenolpyruvic acid		IAF1260:pep		HMDB:HMDB00263		ChemSpider:2907208		ChEBI:58702		PubChem:3674425		CAS:138-08-9		KEGG:C00074	InChI=1S/C3H5O6P/c1-2(3(4)5)9-10(6,7)8/h1H2,(H,4,5)(H2,6,7,8)/p-3
success		BioCyc:PYRUVATE		pyruvate		RefMet:Pyruvic acid		IAF1260:pyr		MetaboLights:MTBLC15361		HMDB:HMDB00243		ChemSpider:96901		PubChem:107735		KNApSAcK:C00001200		ChEBI:15361		KEGG:C00022		CAS:127-17-3		UMBBD-Compounds:c0159	InChI=1S/C3H4O3/c1-2(4)3(5)6/h1H3,(H,5,6)/p-1
success		BioCyc:RIBOSE-5P		D-ribose 5-phosphate		ChEBI:78346		HMDB:HMDB01548		IAF1260:42420		PubChem:24794349		KEGG:C00117		CAS:4300-28-1	InChI=1S/C5H11O8P/c6-1-3(7)5(9)4(8)2-13-14(10,11)12/h1,3-5,7-9H,2H2,(H2,10,11,12)/p-2/t3-,4+,5-/m0/s1
success		BioCyc:RIBULOSE-5P		D-ribulose 5-phosphate		RefMet:Ribulose-5-phosphate		IAF1260:ru5p__D		MetaboLights:MTBLC58121		HMDB:HMDB00618		ChemSpider:20015724		ChEBI:58121		PubChem:21144996		KEGG:C00199		CAS:4151-19-3	InChI=1S/C5H11O8P/c6-1-3(7)5(9)4(8)2-13-14(10,11)12/h4-6,8-9H,1-2H2,(H2,10,11,12)/p-2/t4-,5+/m1/s1
success		BioCyc:D-SEDOHEPTULOSE-7-P		D-sedoheptulose 7-phosphate		RefMet:Sedoheptulose 7-phosphate		IAF1260:s7p		MetaboLights:MTBLC57483		HMDB:HMDB01068		IAF1260:34485		KEGG:C05382		ChEBI:57483		PubChem:46878379		CAS:2646-35-7	InChI=1S/C7H15O10P/c8-1-3(9)5(11)7(13)6(12)4(10)2-17-18(14,15)16/h4-8,10-13H,1-2H2,(H2,14,15,16)/p-2/t4-,5-,6-,7+/m1/s1
ambiguous	BioCyc:Serines		BioCyc:SER	
success		BioCyc:SUC		succinate		RefMet:Succinic acid		IAF1260:succ		MetaboLights:MTBLC30031		HMDB:HMDB00254		ChemSpider:140973		ChEBI:30031		PubChem:160419		KNApSAcK:C00001205		KEGG:C00042		CAS:110-15-6	InChI=1S/C4H6O4/c5-3(6)1-2-4(7)8/h1-2H2,(H,5,6)(H,7,8)/p-2
unknown		Xylulose-5P

Writing metabolite_translation.tab


In [84]:
met_trans = {} 
un_trans = []
with open('Measurements/metabolite_translation.tab') as met_file:
    i = 0
    for line in met_file:
        cols= line.split('\t')
        if cols[0] == 'success':
            met_trans[cols[4]] = dict([col.split(':')
                                       for col in cols[:-1]
                                           if ':' in col])
            met_trans[cols[4]]['InChI'] = cols[-1]
        else:
            un_trans.append(metabolites[i])
        i += 1
met_trans

{'(RS)-malate': {'BioCyc': 'RS-Malate',
  'ChEBI': '15595',
  'InChI': 'InChI=1S/C4H6O5/c5-2(4(8)9)1-3(6)7/h2,5H,1H2,(H,6,7)(H,8,9)/p-2\n'},
 '2-oxoglutarate': {'BioCyc': '2-KETOGLUTARATE',
  'CAS': '328-50-7',
  'ChEBI': '16810',
  'ChemSpider': '144236',
  'HMDB': 'HMDB00208',
  'IAF1260': 'akg',
  'InChI': 'InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(H,7,8)(H,9,10)/p-2\n',
  'KEGG': 'C00026',
  'KNApSAcK': 'C00000769',
  'MetaboLights': 'MTBLC16810',
  'PubChem': '164533',
  'RefMet': 'Oxoglutaric acid'},
 'D-erythrose 4-phosphate': {'BioCyc': 'ERYTHROSE-4P',
  'CAS': '585-18-2',
  'ChEBI': '16897',
  'ChemSpider': '4573609',
  'HMDB': 'HMDB01321',
  'IAF1260': 'e4p',
  'InChI': 'InChI=1S/C4H9O7P/c5-1-3(6)4(7)2-11-12(8,9)10/h1,3-4,6-7H,2H2,(H2,8,9,10)/p-2/t3-,4+/m0/s1\n',
  'KEGG': 'C00279',
  'MetaboLights': 'MTBLC16897',
  'PubChem': '5459862',
  'RefMet': 'Erythrose 4-phosphate'},
 'D-glyceraldehyde 3-phosphate': {'BioCyc': 'GAP',
  'CAS': '591-57-1',
  'ChEBI': '59776',
  'HMD

In [85]:
un_trans, len(met_trans)

(['Arabitol', 'Glucose-6P', 'Isocitrate', 'Serine', 'Xylulose-5P'], 18)